**Tasks 3-4**

Fit a logistic regression model to the given data with the following formula

$$ logit(\pi_i) = \beta_0 \, + \, \beta_1 \, tupak_i \, + \, \beta_2 \, lapsia2_i \, + \, \beta_3 \, lapsia3_i, \: \: \: \: \: \: i=1,...,n. \,$$

Estimate the regression coefficients $\, \boldsymbol{\beta} = (\beta_1, \, \beta_2, \, \beta_3, \, \beta_4)^T \,$ with the **Fisher Scoring** algorithm. 

In [1]:
from statsmodels.formula.api import logit
import pandas as pd
import numpy as np
import scipy

In [2]:
data = pd.read_csv("C:/Users/testi/Desktop/R_GLM2/korvatulehdus_data.csv", index_col=0)

In [3]:
data.shape

(816, 3)

In [4]:
data.head()

,tulehdus,tupak,lapsia
1,1,0,2
2,1,0,2
3,1,1,3
4,0,0,2
5,0,0,2


In [5]:
data.dtypes

tulehdus    int64
tupak       int64
lapsia      int64
dtype: object

In [6]:
data["tupak"] = data["tupak"].astype("category")
data["lapsia"] = data["lapsia"].astype("category")

In [7]:
data.dtypes

tulehdus       int64
tupak       category
lapsia      category
dtype: object

In [8]:
model1 = logit("tulehdus ~ tupak + lapsia", data=data).fit()

Optimization terminated successfully.
         Current function value: 0.671780
         Iterations 5


**Fisher Scoring algorithm**

In [9]:
# Initialize the response variable and the design matrix
y = data["tulehdus"].values
X0 = np.ones(len(y))
X1 = data["tupak"].values
X2 = np.where(data["lapsia"] == 2, 1, 0)
X3 = np.where(data["lapsia"] == 3, 1, 0)
X = pd.DataFrame({"X0": X0,
                  "X1": X1,
                  "X2": X2,
                  "X3": X3})
X = X.values

# Initialize logistic function
def pi(eta):
    return (np.exp(eta) / (1 + np.exp(eta)))

# Initialize beta and epsilon
beta = np.zeros(np.shape(X)[1]).T
eps = 0.001

while eps > 1e-4:
    # Compute the linear predictor
    eta = X @ beta
    
    # Compute the predicted probabilities
    mu = pi(eta)
    
    # Compute diagonal weight matrix
    W = np.diag(mu * (1 - mu))
    
    # Calculate the score function (the first derivative of the log-likelihood) 
    # and the Fisher information matrix
    Score = X.T @ (y - mu)
    Hessian = X.T @ W @ X
    
    # Iteration step
    beta_new = beta + np.linalg.inv(Hessian) @ Score
    
    # Update eps for the while loop threshold criteria
    eps = (beta_new - beta).T @ (beta_new - beta)
    
    # Update beta
    beta = beta_new

Let's build up a dataframe and add two column vectors to it; the estimated $\, \mathbf{\beta} \,$ coefficients that we achieved using the **Fisher Scoring algorithm**, and the corresponding standard errors for the $\, \mathbf{\beta} \,$ coefficients.

Note that the **standard errors** for the **$\, \mathbf{\beta} \,$ coefficients** can be achived by taking the **square root of the diagonal elements of the inverse of the Hessian matrix**.

In [10]:
se = np.sqrt(np.diag(np.linalg.inv(Hessian)))

In [11]:
df = pd.DataFrame({"Beta coeff": beta,
                   "Standard error": se},
                   index=["intercept", "tupak", "lapsia2", "lapsia3"])
df = df.style.set_caption("Fisher Scoring algorithm")

In [12]:
df

,Beta coeff,Standard error
intercept,-0.422455,0.144169
tupak,0.134568,0.196397
lapsia2,0.593833,0.166635
lapsia3,1.322418,0.249863


Now, let's compare these $\, \mathbf{\beta} \,$ coefficients and standard errors derived from the Fisher Scoring algorithm to the $\, \mathbf{\beta} \,$ coefficients and standard errors acquired from fitting the logistic regression model using the **logit** function from the **statsmodels** library.

In [13]:
df2 = pd.DataFrame({"Beta coeff": model1.params.values,
                    "Standard error": model1.bse.values},
                    index=["intercept", "tupak", "lapsia2", "lapsia3"])
df2 = df2.style.set_caption("logit function")

In [14]:
df2

,Beta coeff,Standard error
intercept,-0.422455,0.144169
tupak,0.134568,0.196399
lapsia2,0.593833,0.166635
lapsia3,1.322418,0.249888


- The results are almost identical.

**Task 5**

In [15]:
kids = pd.read_csv("C:/Users/testi/Desktop/R_GLM2/lapset_GLM2.csv", index_col=0)

In [16]:
kids.head()

,aika,apit,apaino,tup,soslk,rviik,parit,sukup,syntpaino
1,46,157,73,1,4,35,12,1,2480
2,27,160,58,0,3,33,1,1,3200
3,42,163,67,0,4,39,5,1,3300
4,32,164,56,0,3,38,1,1,3150
5,27,159,65,0,4,40,1,2,3350


Let's add two new columns (*lapsia* and *kesk*) to the dataframe based on existing columns such that

lapsia: 0, if *parit* = 0 \
$\: \:  \:  \: \: \: \:  \:  \:  \: $ 1, if *parit* = 1 \
$\: \:  \:  \: \: \: \:  \:  \:  \: $ 2, otherwise

kesk: 1, if syntpaino < 2500 \
$\: \: \: \: \:  \:  \:  \: $ 0, otherwise

This can be easily achieved using the **apply function** together with the **lambda function**.

In [17]:
kids["lapsia"] = kids["parit"].apply(lambda x: 0 if x == 0 else 1 if x == 1 else 2)

Let's make sure that the numbers match.

In [18]:
counts = kids["parit"].value_counts()
counts

0     636
1     572
2     355
3     134
4      89
5      60
6      45
7      25
8      15
10      9
9       9
12      5
11      2
13      1
20      1
Name: parit, dtype: int64

In [19]:
counts.loc[0:1]

0    636
1    572
Name: parit, dtype: int64

In [20]:
counts.loc[2:20].sum()

750

In [21]:
kids["lapsia"].value_counts()

2    750
0    636
1    572
Name: lapsia, dtype: int64

In [22]:
kids["kesk"] = kids["syntpaino"].apply(lambda x: 1 if x < 2500 else 0)

To see if the numbers match, let's count the numbers from the original column and from the new column.

In [23]:
kids[kids["syntpaino"] < 2500]["syntpaino"].count()

95

In [24]:
kids[kids["syntpaino"] >= 2500]["syntpaino"].count()

1863

In [25]:
kids["kesk"].value_counts()

0    1863
1      95
Name: kesk, dtype: int64

In [26]:
kids.head()

,aika,apit,apaino,tup,soslk,rviik,parit,sukup,syntpaino,lapsia,kesk
1,46,157,73,1,4,35,12,1,2480,2,1
2,27,160,58,0,3,33,1,1,3200,1,0
3,42,163,67,0,4,39,5,1,3300,2,0
4,32,164,56,0,3,38,1,1,3150,1,0
5,27,159,65,0,4,40,1,2,3350,1,0


**Task 6**

Fit a logistic regression model to the data such that *kesk* is the *output variable* and *tup, lapsia*, *sukup* and the interaction of the variables *tup* and *lapsia* are the input variables. Is the interaction term necessary? Choose the better model and carefully interpret the odds and ratio of odds. Compute the approximate $95 \%$ confidence intervals for the ratio of odds. The regression equation is

$ logit(\pi_{kesk}) = \beta_0 \, + \, \beta_1 \, tup1 \ + \, \beta_2 \, tup2 \, + \, \beta_3 \, lapsia1 \, + \, \beta_4 \, lapsia2 \, + \, \beta_5 \, tup1*lapsia1 \, + \, \beta_6 \, tup2*lapsia1 \\ \: \: \: \: \: \: \: \: \: \: \: \: \: \: \: \: \: \: \: \: \: \: \: \, + \, \beta_7 \, tup1*lapsia2 \, + \, \beta_8 \, tup2*lapsia2 \, + \, \beta_9 \, sukup $

In [27]:
kids["tup"] = kids["tup"].astype("category")
kids["lapsia"] = kids["lapsia"].astype("category")
kids["sukup"] = kids["sukup"].astype("category")

In [28]:
kids[["kesk", "tup", "lapsia", "sukup"]].dtypes

kesk         int64
tup       category
lapsia    category
sukup     category
dtype: object

In [29]:
fit = logit("kesk ~ tup + lapsia + sukup + tup*lapsia", data=kids).fit()

Optimization terminated successfully.
         Current function value: 0.190501
         Iterations 7


In [30]:
fit.params

Intercept              -2.923338
tup[T.1]                0.694852
tup[T.2]                0.614805
lapsia[T.1]            -0.586760
lapsia[T.2]            -0.193458
sukup[T.2]              0.127473
tup[T.1]:lapsia[T.1]   -0.691530
tup[T.2]:lapsia[T.1]   -0.430774
tup[T.1]:lapsia[T.2]   -0.811161
tup[T.2]:lapsia[T.2]    0.469298
dtype: float64

To asses whether the interaction term is needed, one can perform the **deviance test**.

$H_0 :$ "model $\boldsymbol{fit_1}$ is the best model"

$H_1 :$ "model $\boldsymbol{fit_2}$ is the best model"

- where $\, \boldsymbol{fit_1} \,$ is the model without the interaction term, and $\, \boldsymbol{fit_2} \,$ is the same model with the added interaction term between *tup* and *lapsia*.

In [31]:
fit1 = logit("kesk ~ tup + lapsia + sukup", data=kids).fit()

Optimization terminated successfully.
         Current function value: 0.191165
         Iterations 7


In [32]:
fit2 = logit("kesk ~ tup + lapsia + sukup + tup*lapsia", data=kids).fit()

Optimization terminated successfully.
         Current function value: 0.190501
         Iterations 7


The Deviance test statistic is given by

$ D_{test} = D_1 - D_2 \sim \chi^2_{\Delta \, in \, \# \, \beta's} $


where $\, D_1 \,$ and $\, D_2 \,$ are the Deviance residuals for the models $\, \boldsymbol{fit_1} \,$ and $\, \boldsymbol{fit_2}. \,$

The **deviance residual** for a logistic regression model can be calculated as follows

\begin{align}
Deviance_{residual} &= 2(LL(Saturated \, model) - LL(Proposed \, model)) \\
&= 2(-LL(Proposed \, model))
\end{align}

<center>, $where \: \, LL = Log-Likelihood$.</center>

- The reason that the deviance residual equation simplifies to a form seen above is that **the saturated model is expected to have a deviance residual of zero**. This is because the saturated model is a model that includes all the possible predictors and has the **maximum possible fit to the data**. Therefore, it should be able to perfectly predict the response (output) variable for all observations, resulting in a deviance residual of zero.

In the statsmodels library a fitted logistic regression model has an attribute called $\, \boldsymbol{llf}, \,$ which returns the **log-likelihood** of the model.

In [33]:
dev_fit1 = 2*(-fit1.llf)

In [34]:
dev_fit2 = 2*(-fit2.llf)

In [35]:
D_test = dev_fit1 - dev_fit2

In [36]:
D_test

2.6025929720767635

The degrees of freedom is the difference in the number of $\, \boldsymbol{\beta} \,$ coeffiecients between the models.

In [37]:
df = abs(len(fit1.params) - len(fit2.params))
df

4

In [38]:
1 - scipy.stats.chi2.cdf(D_test, df)

0.6263638589971998

- Since the p-value is clearly larger than any commonly used threshold for rejecting the null hypothesis, it can be concluded that $\, \boldsymbol{fit_1} \,$ is the better model and that the interaction term is not needed.

The **AIC** (Akaike Information Criterion) comparison is in line with this conclusion. The smaller the AIC, the better the model, and $\, \boldsymbol{fit_1} \,$ has a smaller AIC value than $\, \boldsymbol{fit_2} \,$.

In [39]:
print(f"fit1 AIC-value: {fit1.aic}")
print(f"fit2 AIC-value: {fit2.aic}")

fit1 AIC-value: 760.6028881255963
fit2 AIC-value: 766.0002951535196


**Interpreting the odds and ratio of odds**

The intercept terms $\, \beta_0 \,$ can be interpreted with odds, and the rest of the $\, \beta \,$ coefficients can be interpreted with a ratio of odds.

In [40]:
b = fit1.params

In [41]:
b

Intercept     -2.858060
tup[T.1]       0.341457
tup[T.2]       0.653431
lapsia[T.1]   -0.752659
lapsia[T.2]   -0.265569
sukup[T.2]     0.121064
dtype: float64

In [42]:
np.exp(b[0])

0.05737995023732782

- It can be inferred from the exponentiated value of $\, \boldsymbol{\beta_0} \,$ that the odds of being a premature baby for a first-born boy with a non-smoking mother is 0.054.

In [43]:
np.exp(b[1])

1.4069954999212184

- It can be inferred from the exponentiated value of $\, \boldsymbol{\beta_1} \,$ that the odds of being a premature baby for a child who's mother smokes a little is 1.401-fold compared to a child who's mother doesn't smoke at all, when adjusting for the other explanatory variables.
- Note that adjusting for the other explanatory variables means that this comparison is eligible only for children who have the same gender and have the same amount of siblings.

In [44]:
np.exp(b[2])

1.9221244481092754

- It can be inferred from the exponentiated value of $\, \boldsymbol{\beta_2} \,$ that the odds of being a premature baby for a child who's mother smokes a lot is 1.922-fold compared to a child who's mother doesn't smoke at all, when adjusting for the other explanatory variables.

In [45]:
np.exp(b[3])

0.4711121088074618

- It can be inferred from the exponentiated value of $\, \boldsymbol{\beta_3} \,$ that the odds of being a premature baby for a second-born child is 0.471-fold compared to a first-born child, when adjusting for the other explanatory variables.

In [46]:
np.exp(b[4])

0.7667698201695687

- It can be inferred from the exponentiated value of $\, \boldsymbol{\beta_4} \,$ that the odds of being a premature baby for a child who's mother has had atleast two pregnancies before is 0.767-fold compared to a first-born child, when adjusting for the other explanatory variables.

In [47]:
np.exp(b[5])

1.1286971108877104

- It can be inferred from the exponentiated value of $\, \boldsymbol{\beta_4} \,$ that the odds of being a premature baby for a girl is 1.129-fold compared to a boy, when adjusting for the other explanatory variables.

$\boldsymbol{95 \%}$ **confidence intervals for the ratio of odds.**

In [48]:
CI = np.exp(fit1.conf_int().iloc[1:, :])

In [49]:
CI_df = pd.DataFrame({"Lower CI": CI[0].values,
                      "Coeff": b.iloc[1:].values,
                      "Upper CI": CI[1].values},
                      index=["tup1", "tup2", "lapsia1", "lapsia2", "sukup[girl]"])

In [50]:
CI_df

,Lower CI,Coeff,Upper CI
tup1,0.776707,0.341457,2.548757
tup2,0.986374,0.653431,3.745601
lapsia1,0.267136,-0.752659,0.830839
lapsia2,0.482190,-0.265569,1.219304
sukup[girl],0.745887,0.121064,1.707976


**Task 7**

In the case of the previous task, present the results using probabilities. If possible, also provide the approximate $\, 95 \% \,$ confidence intervals for the presented probabilities.

In the case of Bernoulli and Binomial distributions, the natural parameter $\, \theta_i \,$ is

$$ \theta_i = log\left(\frac{\pi_i}{1 - \pi_i}\right). $$

solving for $\, \boldsymbol{\pi_i} \,$ gives

$$ \pi_i = \frac{e^{\theta}}{1 + e^{\theta}}, $$

which is called the **inverse logit function**. The inverse logit function gives the probability of the positive outcome as a function of the linear combination of the predictor variables. It is the output of the logistic regression model and represents the estimated probability of the positive outcome given the values of the predictor variables.

In [51]:
fit1 = logit("kesk ~ tup + lapsia + sukup", data=kids).fit()

Optimization terminated successfully.
         Current function value: 0.191165
         Iterations 7


In [52]:
b = fit1.params

In [53]:
def invlogit(theta):
    return (np.exp(theta) / (1 + np.exp(theta)))

In [54]:
b

Intercept     -2.858060
tup[T.1]       0.341457
tup[T.2]       0.653431
lapsia[T.1]   -0.752659
lapsia[T.2]   -0.265569
sukup[T.2]     0.121064
dtype: float64

In [55]:
invlogit(b[0])

0.0542661606402211

- Given that the mother does not smoke, a first-born boy has a probability of 0.054 of being born prematurely.

In [56]:
invlogit(b[1])

0.5845443001315415

- Given that the mother smokes a little, a first-born boy has a probability of 0.585 of being born prematurely. 

In [57]:
invlogit(b[0] + b[5] + b[2] + b[3])

0.05539777408242127

- Given that the mother smokes a lot, a girl who's mother has had one pregnancy before, has a probability of 0.055 of being born prematurely.

In [58]:
invlogit(b[0] + b[5])

0.060825261307114756

- Given that the mother does not smoke, a first-born girl has a probability of 0.061 of being born prematurely.